# Agentic RAG with LlamaIndex


In this notebook we will walk through the basic concepts to create an agents that orchestrate between two query engine tools as follows.

- Define a reader to read the `pdf` sample file [AraGPT2](./data/aragpt2.pdf) paper.
- Define a `splitter` to process the texts of the document.
- Set the LLM embedding and generation model ids.
- Create the engines from the Indexes and define a tool wrapper around them.
- Define a router engine to select the proper tool based on the input query.
- Use the agent to answer some queries.


## Setups


In [1]:
import os
from rich import print
from dotenv import load_dotenv

In [2]:
# load env variables
_ = load_dotenv()

In [3]:
# define some constants
GENERATION_MODEL_ID = "gpt-4o-mini"
EMBEDDING_MODEL_ID = "text-embedding-3-small"

## Load Documents


In [4]:
from llama_index.core import SimpleDirectoryReader

documents_reader = SimpleDirectoryReader(input_files=["./data/aragpt2.pdf"])
documents = documents_reader.load_data()

In [5]:
print(documents[0])

Document(
    id_='b8e65461-168c-4603-a276-a146a13b5fe9',
    embedding=None,
    metadata={
        'page_label': '1',
        'file_name': 'aragpt2.pdf',
        'file_path': 'data/aragpt2.pdf',
        'file_type': 'application/pdf',
        'file_size': 457776,
        'creation_date': '2024-11-12',
        'last_modified_date': '2024-10-22'
    },
    excluded_embed_metadata_keys=[
        'file_name',
        'file_type',
        'file_size',
        'creation_date',
        'last_modified_date',
        'last_accessed_date'
    ],
    excluded_llm_metadata_keys=[
        'file_name',
        'file_type',
        'file_size',
        'creation_date',
        'last_modified_date',
        'last_accessed_date'
    ],
    relationships={},
    text='Proceedings of the Sixth Arabic Natural Language Processing Workshop , pages 196–207\nKyiv, Ukraine 
(Virtual), April 19, 2021.196ARAGPT2: Pre-T rained Transformer\nfor Arabic Language Generation\nWissam Antoun and 
Fady Baly and Hazem Hajj\nAmerican University of Beirut\n{wfa07, fbg06, hh63 }@aub.edu.lb\nAbstract\nRecently, 
pre-trained transformer-based archi-\ntectures have proven to be very efﬁcient atlanguage modeling and 
understanding, giventhat they are trained on a large enough cor-pus. Applications in language generation 
for\nArabic are still lagging in comparison to other\nNLP advances primarily due to the lack ofadvanced Arabic 
language generation models.In this paper, we develop the ﬁrst advanced\nArabic language generation model, 
AraGPT2,\ntrained from scratch on a large Arabic corpusof internet text and news articles. Our largest\nmodel, 
A\nRAGPT2- MEGA , has 1.46 billion pa-\nrameters, which makes it the largest Arabiclanguage model available. 
The\nMEGA model\nwas evaluated and showed success on different\ntasks including synthetic news generation, 
andzero-shot question answering. For text gener-\nation, our best model achieves a perplexity of\n29.8 on held-out 
Wikipedia articles. A studyconducted with human evaluators showed thesigniﬁcant success of AraGPT2-mega in 
gen-erating news articles that are difﬁcult to dis-tinguish from articles written by humans. Wethus develop and 
release an automatic discrim-inator model with a 98% percent accuracy indetecting model-generated text. The 
modelsare also publicly available\n1, hoping to encour-\nage new research directions and applicationsfor Arabic 
NLP.\n1 Introduction\nFew years ago, Natural language processing\n(NLP) was revolutionized with the 
introduction\nof multi-head self-attention transformer architec-ture ( Vaswani et al. ,2017 ). The 
transformer\nachieved superior performance compared to recur-\nrent neural networks several NLP tasks 
including\nmachine translation, sentence classiﬁcation with\n1Pretrained variants of A RAGPT2 (base, 
medium,\nlarge, mega) and discriminator are publicly available 
on\ngithub.com/aub-mind/arabert/tree/master/aragpt2BERT ( Devlin et al. ,2019 ), and ELECTRA ( Clark\net al. ,2020b
), and sentence completion with GPT-\n2(Radford et al. ,2019 ), GROVER ( Zellers et al. ,\n2019 ), and CTRL ( 
Keskar et al. ,2019 ). Recent\nworks have shown that larger models pre-trained\non larger datasets can further 
improve performancei.e. RoBERTa ( Liu et al. ,2019 ), and XLM-R ( Con-\nneau et al. ,2019 ).\nOn the other hand, 
work on Arabic language\nmodeling has mostly targeted natural language\nunderstanding (NLU) by pre-training 
transformer-\nbased models using the Masked Language Model-\ning (MLM) task i.e. A RABERT ( Antoun et al. ,\n2020a 
). In contrast, Arabic text generation or\ncausal language modeling hasn’t received much\nattention. Few works such
as hULMonA ( ElJundi\net al. ,2019 ) used next word prediction as a pre-\ntraining task in for transfer learning in
Arabic text\nclassiﬁcation. ( Khooli ,2020 ) and ( Doiron ,2020 )\nleveraged the existing GPT2 English model 
andadapted it for Arabic using text from the Arabic\nWikipedia dumps, which is sub-optimal for Arabic

In [6]:
from llama_index.core.node_parser import SentenceSplitter

sentence_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=32)
nodes = sentence_splitter.get_nodes_from_documents(documents)

In [7]:
print(len(nodes))
print(nodes[0])

89

TextNode(
    id_='769ef720-fbb9-41e9-86e7-87151b333204',
    embedding=None,
    metadata={
        'page_label': '1',
        'file_name': 'aragpt2.pdf',
        'file_path': 'data/aragpt2.pdf',
        'file_type': 'application/pdf',
        'file_size': 457776,
        'creation_date': '2024-11-12',
        'last_modified_date': '2024-10-22'
    },
    excluded_embed_metadata_keys=[
        'file_name',
        'file_type',
        'file_size',
        'creation_date',
        'last_modified_date',
        'last_accessed_date'
    ],
    excluded_llm_metadata_keys=[
        'file_name',
        'file_type',
        'file_size',
        'creation_date',
        'last_modified_date',
        'last_accessed_date'
    ],
    relationships={
        <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(
            node_id='b8e65461-168c-4603-a276-a146a13b5fe9',
            node_type=<ObjectType.DOCUMENT: '4'>,
            metadata={
                'page_label': '1',
                'file_name': 'aragpt2.pdf',
                'file_path': 'data/aragpt2.pdf',
                'file_type': 'application/pdf',
                'file_size': 457776,
                'creation_date': '2024-11-12',
                'last_modified_date': '2024-10-22'
            },
            hash='be467e8635c423cf5e905a9469550ed6e5194fba229387b29d1ef0cf335d08de'
        ),
        <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(
            node_id='6991976f-1ca4-4766-900b-1ecce132c2a0',
            node_type=<ObjectType.TEXT: '1'>,
            metadata={},
            hash='2d3aa208bc0bae83732ad747eb239cc40eef454a4e693b6422ba4afc8288cdd5'
        )
    },
    text='Proceedings of the Sixth Arabic Natural Language Processing Workshop , pages 196–207\nKyiv, Ukraine 
(Virtual), April 19, 2021.196ARAGPT2: Pre-T rained Transformer\nfor Arabic Language Generation\nWissam Antoun and 
Fady Baly and Hazem Hajj\nAmerican University of Beirut\n{wfa07, fbg06, hh63 }@aub.edu.lb\nAbstract\nRecently, 
pre-trained transformer-based archi-\ntectures have proven to be very efﬁcient atlanguage modeling and 
understanding, giventhat they are trained on a large enough cor-pus. Applications in language generation 
for\nArabic are still lagging in comparison to other\nNLP advances primarily due to the lack ofadvanced Arabic 
language generation models.In this paper, we develop the ﬁrst advanced\nArabic language generation model, 
AraGPT2,\ntrained from scratch on a large Arabic corpusof internet text and news articles. Our largest\nmodel, 
A\nRAGPT2- MEGA , has 1.46 billion pa-\nrameters, which makes it the largest Arabiclanguage model available. 
The\nMEGA model\nwas evaluated and showed success on different\ntasks including synthetic news generation, 
andzero-shot question answering. For text gener-\nation, our best model achieves a perplexity of\n29.8 on held-out 
Wikipedia articles. A studyconducted with human evaluators showed thesigniﬁcant success of AraGPT2-mega in 
gen-erating news articles that are difﬁcult to dis-tinguish from articles written by humans. Wethus develop and 
release an automatic discrim-inator model with a 98% percent accuracy indetecting model-generated text. The 
modelsare also publicly available\n1, hoping to encour-\nage new research directions and applicationsfor Arabic 
NLP.\n1 Introduction\nFew years ago, Natural language processing\n(NLP) was revolutionized with the 
introduction\nof multi-head self-attention transformer architec-ture ( Vaswani et al. ,2017 ).',
    mimetype='text/plain',
    start_char_idx=0,
    end_char_idx=1799,
    text_template='{metadata_str}\n\n{content}',
    metadata_template='{key}: {value}',
    metadata_seperator='\n'
)

## Define Vector and Summary Indexes (Indecies)


In [8]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model=GENERATION_MODEL_ID)
Settings.embed_model = OpenAIEmbedding(model=EMBEDDING_MODEL_ID)

In [9]:
from llama_index.core import VectorStoreIndex, SummaryIndex

vector_index = VectorStoreIndex(nodes=nodes)
summary_index = SummaryIndex(nodes=nodes)

## Define Query Engines

Just tell the engine that there is a tool and here is a short description of how to use it.


In [10]:
vector_query_engine = vector_index.as_query_engine()
summary_query_engine = summary_index.as_query_engine(response_mode="tree_summarize")

In [11]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata


vector_tool = QueryEngineTool(
    query_engine=vector_query_engine,
    metadata=ToolMetadata(
        description="Useful for retrieving specific context from the aragpt2 paper."
    ),
)

summary_tool = QueryEngineTool(
    query_engine=summary_query_engine,
    metadata=ToolMetadata(
        description="Useful for summarization questions related to the aragpt2 paper."
    ),
)

## Define Router Query Engine


In [12]:
from llama_index.core.selectors import LLMSingleSelector
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine

agent = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[vector_tool, summary_tool],
    verbose=True,
)

In [13]:
query = "Summarize the AraGPT2 paper for me please."
response = agent.query(query)

Selecting query engine 1: The question specifically asks for a summary of the AraGPT2 paper, making choice 2, which is useful for summarization questions, the most relevant..


In [18]:
print(response.response)

The AraGPT2 paper presents the development of the first advanced Arabic language generation model, trained from 
scratch on a large corpus of Arabic text, including internet articles and news. The model, AraGPT2, comes in four 
variants: base, medium, large, and mega, with the largest variant containing 1.46 billion parameters, making it the
largest Arabic language model available.

The paper highlights the model's effectiveness in various tasks, such as synthetic news generation and zero-shot 
question answering, achieving a perplexity score of 29.8 on held-out Wikipedia articles. Human evaluations 
demonstrated that AraGPT2-Mega can generate news articles that are often indistinguishable from those written by 
humans, successfully fooling approximately 60% of evaluators.

Additionally, the authors developed an automatic discriminator model that can detect machine-generated text with 
98% accuracy. The paper emphasizes the importance of making such models publicly available to foster research in 
Arabic NLP, while also addressing ethical concerns regarding potential misuse, such as generating fake news. The 
findings and methodologies outlined in the paper aim to encourage further exploration and applications in the field
of Arabic natural language processing.

In [19]:
query = "What data used to train the AraGPT2 model?"
response = agent.query(query)

Selecting query engine 0: The question asks for specific context regarding the data used to train the AraGPT2 model, which aligns with retrieving specific information..


In [20]:
print(response.response)

The training dataset for the AraGPT2 model includes the following publicly available Arabic corpora:

- The unshuffled OSCAR corpus
- The Arabic Wikipedia dump from September 2020
- The 1.5B words Arabic Corpus
- The OSIAN corpus
- News articles provided by As-safir newspaper

Additionally, the dataset underwent preprocessing, which involved filtering out short documents, removing repeated 
sentences, and replacing URLs, emails, and user mentions with special tokens, among other modifications.